This is just a simple test notebook for dataset_pixel.py.

In [1]:
# Cell 1: Imports & project setup
import sys
from pathlib import Path

# Adjust this if the notebook is located elsewhere;
# here we assume notebook is one level under project root.
project_root = Path(__file__).resolve().parent.parent if '__file__' in globals() else Path().resolve().parent
sys.path.append(str(project_root))

import torch
import numpy as np
import cv2
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader

from common.config import DEVICE, WAVES_DIR, SECTIONS_DIR, RAYS_DIR
from common.pmatrix import tensor_pmatrix
from common.waveforms import load_all_waveforms
from common.dmatrix import load_ray_tensor
from models.autoencoder import ConvAutoencoder, DatasetAutoencoder
from models.pixel_nn.dataset_pixel import PixelDataset, index_to_triplet, pixel_bounds


config.py: DEVICE is set as cuda


In [2]:
# Cell 2: Load your autoencoder
# ─────────────────────────────────
# Adjust CKPT_PATH to point to your trained AE checkpoint
CKPT_PATH = project_root / "artifacts/autoencoder/checkpoints/ConvAE_resample200_lat32_bn_2025-06-20_09-56-52.pt"

# Create a DatasetAutoencoder for initialization
wave_ds = DatasetAutoencoder(
    path=Path(WAVES_DIR),
    reduction="resample",
    n=200,
    save=False,
    force_reload=False
)

# Load the AE and switch to eval mode
ae = ConvAutoencoder.load(
    path=CKPT_PATH,
    dataset=wave_ds,
    device=DEVICE
).to(DEVICE)
ae.eval()

print("✅ Loaded autoencoder:", ae)

✅ Loaded autoencoder: ConvAutoencoder(
  (encoder): Sequential(
    (0): Conv1d(1, 8, kernel_size=(15,), stride=(5,), padding=(7,))
    (1): BatchNorm1d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ELU(alpha=1.0)
    (3): Conv1d(8, 16, kernel_size=(5,), stride=(2,), padding=(2,))
    (4): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ELU(alpha=1.0)
    (6): Flatten(start_dim=1, end_dim=-1)
    (7): Linear(in_features=320, out_features=1024, bias=True)
    (8): ELU(alpha=1.0)
    (9): Linear(in_features=1024, out_features=32, bias=True)
  )
  (decoder): Sequential(
    (0): Linear(in_features=32, out_features=1024, bias=True)
    (1): ELU(alpha=1.0)
    (2): Linear(in_features=1024, out_features=320, bias=True)
    (3): ELU(alpha=1.0)
    (4): Unflatten(dim=1, unflattened_size=(16, 20))
    (5): ConvTranspose1d(16, 8, kernel_size=(5,), stride=(2,), padding=(2,), output_padding=(1,))
    (6): BatchNorm1d(8, eps=1e-

In [3]:
# Cell 3: Instantiate PixelDataset
# ─────────────────────────────────
pixel_ds = PixelDataset(
    autoencoder=ae,
    nX=30,
    nY=20,
    save=False   # disable saving during tests
)

print("✅ PixelDataset created")
print("  - # sections:", pixel_ds.num_sections)
print("  - Total pixels:", len(pixel_ds))

Encoded waveforms with shape torch.Size([6600, 32]) (num_waveforms, encoding_dims)
Sorted files in rays
Saved ray tensors shape: torch.Size([100, 66, 30, 20]) (num_sections=100, num_rays=66, nX, nY)
C:\Users\Jaime\Documents\deep-learning-wall-tomography-2\data\sections
<generator object Path.glob at 0x0000021048994200>
Loaded 100 section files with labels shape: torch.Size([100, 20, 30]) (num_sections, nY, nX)
✅ PixelDataset created
  - # sections: 100
  - Total pixels: 60000


In [5]:
# Cell 4: Basic sanity checks
# ─────────────────────────────────
# 1) Length
expected_len = pixel_ds.num_sections * pixel_ds.nX * pixel_ds.nY
print("Expected total pixels:", expected_len)
print("len(pixel_ds):", len(pixel_ds))

# 2) First element
features0, label0 = pixel_ds[0]
print("features[0].shape:", features0.shape)
print("label[0]:", label0)

# 4) Test slice
batch_feats, batch_labels = pixel_ds[:5]
print("Batch features shape (first 5):", batch_feats.shape)
print("Batch labels shape (first 5):", batch_labels.shape)


Expected total pixels: 60000
len(pixel_ds): 60000
features[0].shape: torch.Size([2182])
label[0]: tensor(0.)
Batch features shape (first 5): torch.Size([5, 2182])
Batch labels shape (first 5): torch.Size([5])


C:\Users\Jaime\Documents\deep-learning-wall-tomography-2\models\pixel_nn\dataset_pixel.py:202: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label = torch.tensor(self.labels[p, y, x])


In [6]:
# Cell 5: Consistency checks
enc_dim = pixel_ds.encoded_waveforms.size(1)
num_rays = pixel_ds.ray_tensors.size(1)

for idx in [0, 10, 500, len(pixel_ds) - 10]:
    features, label = pixel_ds[idx]
    p, y, x = index_to_triplet(idx, pixel_ds.nX, pixel_ds.nY)

    # Compute true bounds
    true_bounds = pixel_bounds(
        x, y,
        pixel_ds.nX, pixel_ds.nY,
        pixel_ds.Xmin, pixel_ds.Xmax,
        pixel_ds.Ymin, pixel_ds.Ymax
    )
    feat_bounds = features[:4]

    # Extract ray distances from feature vector
    start = 4
    length_per_ray = 1 + enc_dim
    ray_block = features[start : start + num_rays * length_per_ray]
    ray_block = ray_block.view(num_rays, length_per_ray)
    feat_distances = ray_block[:, 0]

    # Fetch expected distances
    expected_distances = pixel_ds.ray_tensors[p, :, x, y]

    # Fetch expected label
    expected_label = pixel_ds.labels[p, y, x]

    # Log everything
    print(f"\n--- Index {idx} (section={p}, y={y}, x={x}) ---")
    print("Feature bounds:     ", feat_bounds.numpy())
    print("True bounds:        ", true_bounds.numpy())
    print("Bounds match?       ", np.allclose(feat_bounds.numpy(), true_bounds.numpy()))

    print("\nFirst 5 feature distances:", feat_distances[:5].numpy())
    print("First 5 expected distances:", expected_distances[:5].numpy())
    print("Distances match?        ", np.allclose(feat_distances.numpy(), expected_distances.numpy()))

    print(f"\nFeature label:    {label.item()}")
    print(f"Expected label:   {expected_label}")
    print("Label match?      ", label.item() == expected_label)

print("\n✅ Logged consistency checks for selected indices.")



--- Index 0 (section=0, y=0, x=0) ---
Feature bounds:      [0.   0.   0.02 0.02]
True bounds:         [0.   0.   0.02 0.02]
Bounds match?        True

First 5 feature distances: [0. 0. 0. 0. 0.]
First 5 expected distances: [0. 0. 0. 0. 0.]
Distances match?         True

Feature label:    0.0
Expected label:   0.0
Label match?       tensor(True)

--- Index 10 (section=0, y=0, x=10) ---
Feature bounds:      [0.2  0.   0.22 0.02]
True bounds:         [0.2  0.   0.22 0.02]
Bounds match?        True

First 5 feature distances: [0.         0.         0.         0.04272002 0.02236068]
First 5 expected distances: [0.         0.         0.         0.04272002 0.02236068]
Distances match?         True

Feature label:    1.0
Expected label:   1.0
Label match?       tensor(True)

--- Index 500 (section=0, y=16, x=20) ---
Feature bounds:      [0.4  0.32 0.42 0.34]
True bounds:         [0.4  0.32 0.42 0.34]
Bounds match?        True

First 5 feature distances: [0. 0. 0. 0. 0.]
First 5 expected dista

In [7]:
# Cell 6: Quick visual inspection
# ─────────────────────────────────
idx = 0
feat, lbl = pixel_ds[idx]
print(f"Feature vector (first 100 entries) @ idx={idx}:", feat[:100])
print(f"Label @ idx={idx}:", lbl)


Feature vector (first 100 entries) @ idx=0: tensor([ 0.0000e+00,  0.0000e+00,  2.0000e-02,  2.0000e-02,  0.0000e+00,
         5.2062e+01,  1.7689e+01,  1.1608e+02, -1.2524e+01,  1.1628e+01,
         4.4147e+01, -9.5998e+00,  3.0363e+01, -3.3254e+01,  4.7424e+01,
        -2.3140e+01,  1.0771e+02, -7.7548e+01,  2.2487e+01,  1.0268e+02,
        -5.0443e+01, -9.2029e+01, -9.7928e+01,  1.5053e+02, -2.7209e+01,
         2.5511e+01, -3.4609e+01, -4.6145e+01,  1.5202e+01,  1.2379e+01,
        -1.0710e+00, -6.7435e+01,  1.1358e+01,  4.7720e+01,  2.7765e+01,
        -4.9607e+01, -3.4104e+01,  0.0000e+00,  1.3982e+00, -6.2443e+00,
         3.3082e+01,  7.6359e+00,  9.4067e+00,  4.4811e+01, -2.8527e+01,
         1.0355e+02, -4.7832e+01,  1.0069e+02, -2.8791e+01,  1.1360e+02,
        -8.2140e+01, -3.0420e+01,  9.7028e+01, -2.9963e+01,  3.0240e+00,
        -7.3859e+01,  1.4843e+02, -7.7909e+01,  8.1853e+01,  1.9065e+00,
         3.9287e+01, -2.4087e+01,  2.8214e+00, -1.5985e+01,  5.7373e+00,
       